In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
time_series = pd.read_csv('../data/no_consec.csv')
time_series = time_series.drop(columns = {'Unnamed: 0', 
                                          'index',
                                          'RecordID',
                                          'ESS_Id',
                                          'ecuSoftwareVersion', 
                                          'ecuSerialNumber',
                                          'ecuModel',
                                          'ecuMake',
                                          'ecuSource',
                                          'actionDescription',
                                          'active', 
                                          'activeTransitionCount', 
                                          'faultValue', 
                                          'MCTNumber'})

C:\Users\hjrko\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
problem_cols = ['AcceleratorPedal',
 'BarometricPressure',
 'CruiseControlActive',
 'CruiseControlSetSpeed',
 'DistanceLtd',
 'EngineCoolantTemperature',
 'EngineLoad',
 'EngineOilPressure',
 'EngineOilTemperature',
 'EngineRpm',
 'EngineTimeLtd',
 'FuelLevel',
 'FuelLtd',
 'FuelRate',
 'FuelTemperature',
 'IgnStatus',
 'IntakeManifoldTemperature',
 'LampStatus',
 'ParkingBrake',
 'ServiceDistance',
 'Speed',
 'SwitchedBatteryVoltage',
 'Throttle',
 'TurboBoostPressure']

for x in problem_cols:
    try:
        time_series[x] = time_series[x].astype(str).str.replace(',','').astype(float)
    except ValueError:
        time_series[x] = time_series[x]

In [4]:
time_series['EventTimeStamp'] = time_series['EventTimeStamp'].astype('datetime64[ns]')
time_series['LocationTimeStamp'] = time_series['LocationTimeStamp'].astype('datetime64[ns]')

In [7]:
time_series['EquipmentID'] = time_series['EquipmentID'].astype(str)

In [5]:
time_series = time_series.sort_values(['EquipmentID','EventTimeStamp']).reset_index().drop(columns=['index'])

Create a row that has encoding of spn seqeunces. Ex. 111 leads to ??? spn which eventually leads to derate. Use this as an indicator on if a model will derate<br>
Use bit flags (different columns) to determine if a derate will happen or not

In [19]:
time_series[(time_series['spn'] == '305')]

,EventTimeStamp,eventDescription,spn,fmi,EquipmentID,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,...,TurboBoostPressure,derate,full_derate,derate_cumulative,derate_cumu_diff,next_timestamp,time_diff_next,time_diff_next_h,updated_time,new_time_diff
7522,2015-02-21 12:18:36,Special Instructions System Diagnostic Code #1,611,14,305,33.673379,-84.340185,2015-02-21 12:18:32,NaN,NaN,...,NaN,0,0,390,0.0,2020-01-17 18:59:41,-1792 days +17:18:55,0.000000,0.000000,7110.116667
7523,2015-02-21 12:18:36,Special Instructions Wheel Sensor ABS Axle 2 Left,791,14,305,33.673379,-84.340185,2015-02-21 12:18:32,NaN,NaN,...,NaN,0,0,390,0.0,2015-02-21 12:18:36,0 days 00:00:00,0.000000,0.000000,7110.116667
7524,2015-02-21 12:18:38,Special Instructions System Diagnostic Code #1,611,14,305,33.673750,-84.340324,2015-02-21 12:19:14,NaN,NaN,...,NaN,0,0,390,0.0,2015-02-21 12:18:36,0 days 00:00:02,0.000556,0.000556,7110.116667
7525,2015-02-21 12:18:38,Special Instructions Wheel Sensor ABS Axle 2 Left,791,14,305,33.673750,-84.340324,2015-02-21 12:19:14,NaN,NaN,...,NaN,0,0,390,0.0,2015-02-21 12:18:38,0 days 00:00:00,0.000000,0.000000,7110.116111
7526,2015-02-21 12:19:17,Data May Be Invalid Relative Speed; Rear Axle ...,907,19,305,33.673750,-84.340324,2015-02-21 12:19:13,NaN,NaN,...,NaN,0,0,390,0.0,2015-02-21 12:18:38,0 days 00:00:39,0.010833,0.010833,7110.116111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12451,2020-01-24 08:17:55,Error in System Engine Exhaust Gas Recirculati...,2791,11,305,35.485231,-86.076759,2020-01-24 08:17:50,NaN,NaN,...,NaN,0,0,403,0.0,2020-01-24 08:17:55,0 days 00:00:00,0.000000,0.000000,0.000000
12452,2020-02-27 12:03:29,High Voltage (Left Fuel Level Sensor),829,3,305,35.490185,-86.410324,2020-02-27 12:04:06,0.0,14.4275,...,0.29,0,0,403,0.0,2020-01-24 08:17:55,34 days 03:45:34,819.759444,0.000000,0.000000
12453,2020-02-27 12:03:29,High Voltage (Fuel Level),96,3,305,35.490185,-86.410324,2020-02-27 12:04:06,0.0,14.4275,...,0.29,0,0,403,0.0,2020-02-27 12:03:29,0 days 00:00:00,0.000000,0.000000,0.000000
12454,2020-02-27 12:08:48,High Voltage (Fuel Level),96,3,305,35.491018,-86.409583,2020-02-27 12:08:44,NaN,NaN,...,NaN,0,0,403,0.0,2020-02-27 12:03:29,0 days 00:05:19,0.088611,0.000000,0.000000
